In [1]:
import os
import torch
import shutil
import random
import warnings
import boto3
import s3fs
import io
import time
import botocore.exceptions

import pandas as pd
import numpy as np
# import torch.nn.functional as F
import getpass
import json

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import transforms, utils, models, datasets
from torch import nn, optim
from torch.optim import lr_scheduler
from io import BytesIO
from tqdm import tqdm
from skimage import io, transform
from PIL import Image
from PIL import Image
from pycocotools import mask as maskUtils
from tqdm.notebook import tqdm

warnings.filterwarnings("ignore")


import fiftyone as fo
import fiftyone.brain as fob
import fiftyone.zoo as foz
from fiftyone import ViewField as F

In [4]:
access_key = getpass.getpass("Enter your access: ")

secret_key = password = getpass.getpass("Enter your secret: ")

Enter your access:  ········
Enter your secret:  ········


In [5]:
bucket_name = 'w210facetdata'
annotations_prefix = 'annotations/'
images_prefix = '/home/ubuntu/W210-Capstone'

In [47]:
s3 = s3fs.S3FileSystem(key=access_key, secret=secret_key)

# Use s3.open to open the CSV file and read its content into a Pandas DataFrame
with s3.open(f's3://{bucket_name}/{annotations_prefix}annotations.csv', 'rb') as file:
    gt_df = pd.read_csv(file)

In [2]:
## use relative paths to your image dirs
# dataset = fo.Dataset(name = "FACET14", persistent=True)
dataset = fo.load_dataset('FACET14')
# dataset.add_images_dir(images_prefix)
dataset.compute_metadata()

# Object Detection Functions

In [34]:
BOOLEAN_PERSONAL_ATTRS = (
    "has_facial_hair",
    "has_tattoo",
    "has_cap",
    "has_mask",
    "has_headscarf",
    "has_eyeware",
)
def add_boolean_person_attributes(detection, row_index):
    for attr in BOOLEAN_PERSONAL_ATTRS:
        detection[attr] = gt_df.loc[row_index, attr].astype(bool)

In [35]:
def get_hairtype(row_index):
    hair_info = gt_df.loc[row_index, gt_df.columns.str.startswith('hairtype')]
    hairtype = hair_info[hair_info == 1]
    if len(hairtype) == 0:
        return None
    return hairtype.index[0].split('_')[1]

def get_haircolor(row_index):
    hair_info = gt_df.loc[row_index, gt_df.columns.str.startswith('hair_color')]
    haircolor = hair_info[hair_info == 1]
    if len(haircolor) == 0:
        return None
    return haircolor.index[0].split('_')[2]

In [36]:
def add_person_attributes(detection, row_index):
    detection["hairtype"] = get_hairtype(row_index)
    detection["haircolor"] = get_haircolor(row_index)
    add_boolean_person_attributes(detection, row_index)

In [37]:
def get_perceived_gender_presentation(row_index):
    gender_info = gt_df.loc[row_index, gt_df.columns.str.startswith('gender')]
    pgp = gender_info[gender_info == 1]
    if len(pgp) == 0:
        return None
    return pgp.index[0].replace("gender_presentation_", "").replace("_", " ")

def get_perceived_age_presentation(row_index):
    age_info = gt_df.loc[row_index, gt_df.columns.str.startswith('age')]
    pap = age_info[age_info == 1]
    if len(pap) == 0:
        return None
    return pap.index[0].split('_')[2]

In [38]:
def get_skintone(row_index):
    skin_info = gt_df.loc[row_index, gt_df.columns.str.startswith('skin_tone')]
    return skin_info.to_dict()

In [39]:
def add_protected_attributes(detection, row_index):
    detection["perceived_age_presentation"] = get_perceived_age_presentation(row_index)
    detection["perceived_gender_presentation"] = get_perceived_gender_presentation(row_index)
    detection["skin_tone"] = get_skintone(row_index)

In [40]:
VISIBILITY_ATTRS = ("visible_torso", "visible_face", "visible_minimal")

In [41]:
def get_lighting(row_index):
    lighting_info = gt_df.loc[row_index, gt_df.columns.str.startswith('lighting')]
    lighting = lighting_info[lighting_info == 1]
    if len(lighting) == 0:
        return None
    lighting = lighting.index[0].replace("lighting_", "").replace("_", " ")
    return lighting

def add_other_attributes(detection, row_index):
    detection["lighting"] = get_lighting(row_index)
    for attr in VISIBILITY_ATTRS:
        detection[attr] = gt_df.loc[row_index, attr].astype(bool)

In [42]:
def create_detection(row_index, sample):
    bbox_dict = json.loads(gt_df.loc[row_index, "bounding_box"])
    x, y, w, h = bbox_dict["x"], bbox_dict["y"], bbox_dict["width"], bbox_dict["height"]
    cat1, cat2 = bbox_dict["dict_attributes"]["cat1"], bbox_dict["dict_attributes"]["cat2"]

    person_id = gt_df.loc[row_index, "person_id"]

    img_width, img_height = sample.metadata.width, sample.metadata.height

    bounding_box = [x/img_width, y/img_height, w/img_width, h/img_height]
    detection = fo.Detection(
        label=cat1, 
        bounding_box=bounding_box,
        person_id=person_id,
        )
    if cat2 != 'none':
        detection["class2"] = cat2

    add_person_attributes(detection, row_index)
    add_protected_attributes(detection, row_index)
    add_other_attributes(detection, row_index)

    return detection

In [43]:
def add_ground_truth_labels(dataset):
    for sample in dataset.iter_samples(autosave=True, progress=True):
        sample_annos = gt_df[gt_df['filename'] == sample.filename]
        detections = []
        for row in sample_annos.iterrows():
            row_index = row[0]
            detection = create_detection(row_index, sample)
            detections.append(detection)
        sample["ground_truth"] = fo.Detections(detections=detections)
    dataset.add_dynamic_sample_fields()



In [162]:
dataset.

Name:        FACET14
Media type:  image
Num samples: 31702
Persistent:  True
Tags:        []
Sample fields:
    id:              fiftyone.core.fields.ObjectIdField
    filepath:        fiftyone.core.fields.StringField
    tags:            fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:        fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth:    fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_masks:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    yolov5m:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_yolov5m_tp: fiftyone.core.fields.IntField
    eval_yolov5m_fp: fiftyone.core.fields.IntField
    eval_yolov5m_fn: fiftyone.core.fields.IntField

# Add labels

In [48]:
## add all of the ground truth labels
add_ground_truth_labels(dataset)

 100% |█████████████| 31702/31702 [5.5m elapsed, 0s remaining, 93.8 samples/s]       


# Add Masks

In [50]:
def add_coco_masks_to_dataset(dataset):
    with s3.open(f's3://{bucket_name}/{annotations_prefix}coco_masks.json', 'rb') as file:
        coco_masks = json.load(file)
    cmas = coco_masks["annotations"]

    FILENAME_TO_ID = {
        img["file_name"]: img["id"]
        for img in coco_masks["images"]
    }

    CAT_TO_LABEL = {cat["id"]: cat["name"] for cat in coco_masks["categories"]}

    for sample in dataset.iter_samples(autosave=True, progress=True):
        fn = sample.filename

        if fn not in FILENAME_TO_ID:
            continue

        img_id = FILENAME_TO_ID[fn]
        img_width, img_height = sample.metadata.width, sample.metadata.height
        sample_annos = [a for a in cmas if a["image_id"] == img_id]
        if len(sample_annos) == 0:
            continue

        coco_detections = []
        for ann in sample_annos:
            label = CAT_TO_LABEL[ann["category_id"]]
            bbox = ann['bbox']
            ann_id = ann['ann_id']
            person_id = ann['facet_person_id']

            mask = maskUtils.decode(ann["segmentation"])
            mask = Image.fromarray(255*mask)

            ## Change bbox to be in the format [x, y, x, y]
            bbox[2] = bbox[0] + bbox[2]
            bbox[3] = bbox[1] + bbox[3]

            ## Get the cropped image
            cropped_mask = np.array(mask.crop(bbox)).astype(bool)

            ## Convert to relative [x, y, w, h] coordinates
            bbox[2] = bbox[2] - bbox[0]
            bbox[3] = bbox[3] - bbox[1]

            bbox[0] = bbox[0]/img_width
            bbox[1] = bbox[1]/img_height
            bbox[2] = bbox[2]/img_width
            bbox[3] = bbox[3]/img_height

            new_detection = fo.Detection(
                label=label, 
                bounding_box=bbox,
                person_id=person_id,
                ann_id=ann_id,
                mask=cropped_mask,
                )
            coco_detections.append(new_detection)
        sample["coco_masks"] = fo.Detections(detections=coco_detections)

## add the masks
add_coco_masks_to_dataset(dataset)

 100% |█████████████| 31702/31702 [26.0m elapsed, 0s remaining, 16.5 samples/s]      


# Import Yolo

In [51]:
yolov5 = foz.load_zoo_model('yolov5m-coco-torch')

Using cache found in /home/ubuntu/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2023-10-21 Python-3.10.9 torch-2.1.0+cu121 CUDA:0 (Tesla T4, 15102MiB)

100%|███████████████████████████████████████| 40.8M/40.8M [00:00<00:00, 143MB/s]

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


In [52]:
dataset.apply_model(yolov5, label_field="yolov5m")
### Just retain the "person" detections
people_view_values = dataset.filter_labels("yolov5m", F("label") == "person").values("yolov5m")
dataset.set_values("yolov5m", people_view_values)
dataset.save()

WARNING ⚠️ NMS time limit 0.550s exceeded


 100% |█████████████| 31702/31702 [24.5m elapsed, 0s remaining, 21.9 samples/s]      


# Clip classification model --> Replace with teacher/student

In [54]:
## get a list of all 52 classes
facet_classes = dataset.distinct("ground_truth.detections.label")

## instantiate a CLIP model with these classes
clip = foz.load_zoo_model(
    "clip-vit-base32-torch",
    text_prompt="A photo of a",
    classes=facet_classes,
)

In [55]:
patch_view = dataset.to_patches("ground_truth")
patch_view.apply_model(clip, label_field="clip")
dataset.save_view("patch_view", patch_view)

 100% |█████████████| 49551/49551 [24.6m elapsed, 0s remaining, 34.3 samples/s]      


In [56]:
IOU_THRESHS = np.round(np.arange(0.5, 1.0, 0.05), 2)

In [57]:
def _evaluate_detection_model(dataset, label_field):
    eval_key = "eval_" + label_field.replace("-", "_")
    dataset.evaluate_detections(label_field, "ground_truth", eval_key=eval_key, classwise=False)
    
    for sample in dataset.iter_samples(progress=True):
        for pred in sample[label_field].detections:
            iou_field = f"{eval_key}_iou"
            if iou_field not in pred:
                continue

            iou = pred[iou_field]
            for it in IOU_THRESHS:
                pred[f"{iou_field}_{str(it).replace('.', '')}"] = iou >= it
        sample.save()

In [58]:
_evaluate_detection_model(dataset, 'yolov5m')

Evaluating detections...
 100% |█████████████| 31702/31702 [4.6m elapsed, 0s remaining, 65.7 samples/s]       
 100% |█████████████| 31702/31702 [8.1m elapsed, 0s remaining, 49.7 samples/s]       


In [59]:
def _compute_detection_mAR(sample_collection, label_field):
    """Computes the mean average recall of the specified detection field.
    -- computed as the average over iou thresholds of the recall at
    each threshold.
    """
    eval_key = "eval_" + label_field.replace("-", "_")
    iou_recalls = []
    for it in IOU_THRESHS:
        field_str = f"{label_field}.detections.{eval_key}_iou_{str(it).replace('.', '')}"
        counts = sample_collection.count_values(field_str)
        tp, fn = counts.get(True, 0), counts.get(False, 0)
        recall = tp/float(tp + fn) if tp + fn > 0 else 0.0
        iou_recalls.append(recall)

    return np.mean(iou_recalls)

In [92]:
def get_concept_attr_detection_mAR(dataset, label_field, concept, attributes):
    sub_view = dataset.filter_labels("ground_truth", F("label") == concept)
    for attribute in attributes.items():
        if "skin_tone" in attribute[0]:
            sub_view = sub_view.filter_labels("ground_truth", F(f"skin_tone.{attribute[0]}") != 0)
        else:
            sub_view = sub_view.filter_labels("ground_truth", F(attribute[0]) == attribute[1])
    return _compute_detection_mAR(sub_view, label_field)

In [93]:
concept = 'lawman'
attributes = {"hairtype": "straight", "haircolor": "brown"}
get_concept_attr_detection_mAR(dataset, "yolov5m", concept, attributes)


0.4259259259259259

In [136]:
def _evaluate_classification_model(dataset, prediction_field):
    patch_view = dataset.load_saved_view("patch_view")
    eval_key = "eval_" + prediction_field
    
    for sample in patch_view.iter_samples(progress=True):
        sample[eval_key] = (
            sample.ground_truth.label == sample[prediction_field].label
        )
        sample.save()
    dataset.save_view("patch_view", patch_view, overwrite=True)

In [137]:
_evaluate_classification_model(dataset, 'clip')

 100% |█████████████| 49551/49551 [2.5m elapsed, 0s remaining, 329.3 samples/s]      


In [21]:
def _compute_classification_recall(patch_collection, label_field):
    eval_key = "eval_" + label_field.split("_")[0]
    counts = patch_collection.count_values(eval_key)
    tp, fn = counts.get(True, 0), counts.get(False, 0)
    recall = tp/float(tp + fn) if tp + fn > 0 else 0.0
    return recall

In [22]:
def get_concept_attr_classification_recall(dataset, label_field, concept, attributes):
    patch_view = dataset.load_saved_view("patch_view")
    sub_patch_view = patch_view.match(F("ground_truth.label") == concept)
    for attribute in attributes.items():
        if "skin_tone" in attribute[0]:
            sub_patch_view = sub_patch_view.match(F(f"ground_truth.skin_tone.{attribute[0]}") != 0)
        else:
            sub_patch_view = sub_patch_view.match(F(f"ground_truth.{attribute[0]}") == attribute[1])
    return _compute_classification_recall(sub_patch_view, label_field)

In [23]:
attribute = {'hairtype': 'curly'}

In [24]:
get_concept_attr_classification_recall(dataset, "clip", concept, attribute)


0.0

In [25]:
def get_concept_attr_recall(dataset, label_field, concept, attribute):
    if label_field in dataset.get_field_schema().keys():
        return get_concept_attr_detection_mAR(dataset, label_field, concept, attribute)
    else:
        return get_concept_attr_classification_recall(dataset, label_field, concept, attribute)

In [26]:
get_concept_attr_classification_recall(dataset, 'clip', 'singer', attrs1)

0.5362318840579711

In [27]:
def compute_disparity(dataset, label_field, concept, attribute1, attribute2):
    recall1 = get_concept_attr_recall(dataset, label_field, concept, attribute1)
    recall2 = get_concept_attr_recall(dataset, label_field, concept, attribute2)
    return recall1 - recall2

In [28]:
attrs1 = {"perceived_gender_presentation": "fem"}
attrs2 = {"hairtype": "straight"}
for concept in ["astronaut", "singer", "judge", "student"]:
    disparity = compute_disparity(dataset, "clip", concept, attrs1, attrs2)     
    print(f"{concept}: {disparity}")

astronaut: -0.8269230769230769
singer: -0.0008051529790660261
judge: -0.06666666666666667
student: 0.16279069767441856


# Experimenting with uploading custom models

In [86]:
import fiftyone.utils.torch as fout
from torchvision.models import resnet50
import torchvision

In [87]:
model = resnet50(pretrained=True)
num_new_classes = 52

# Modify the final fully connected (fc) layer
# Get the number of input features to the fc layer
in_features = model.fc.in_features

# Replace the fc layer with a new one for the desired number of classes
model.fc = nn.Linear(in_features, num_new_classes)

def make_data_loader(image_paths, sample_ids, batch_size):
    mean = [0.4914, 0.4822, 0.4465]
    std = [0.2023, 0.1994, 0.2010]
    transforms = torchvision.transforms.Compose(
        [
            torchvision.transforms.Resize((256, 256)),
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean, std),
        ]
    )
    dataset = fout.TorchImageDataset(
        image_paths, sample_ids=sample_ids, transform=transforms
    )
    return DataLoader(dataset, batch_size=batch_size, num_workers=4)

In [88]:
def predict(model, imgs):
    logits = model(imgs).detach().cpu().numpy()
    predictions = np.argmax(logits, axis=1)
    odds = np.exp(logits)
    confidences = np.max(odds, axis=1) / np.sum(odds, axis=1)
    return predictions, confidences

In [89]:
num_samples = 1000
batch_size = 5

view = dataset.take(num_samples, seed=51)
classes = []
for i in view.iter_samples():
    classes.append(i.ground_truth.detections[0].label)

image_paths, sample_ids = zip(*[(s.filepath, s.id) for s in view])
data_loader = make_data_loader(image_paths, sample_ids, batch_size)

#
# Perform prediction and store results in dataset
#

for imgs, sample_ids in data_loader:
    predictions, confidences = predict(model, imgs)

    # Add predictions to your FiftyOne dataset
    for sample_id, prediction, confidence in zip(
        sample_ids, predictions, confidences
    ):
        sample = dataset[sample_id]
        sample["resnet_pred"] = fo.Classification(
            label=classes[prediction],
            confidence=confidence,
        )
        sample.save()



In [90]:
def _evaluate_classification_modelr(dataset, prediction_field):
    eval_key = "eval_" + prediction_field
    
    for sample in dataset.iter_samples(progress=True):
        sample[eval_key] = (
            sample.ground_truth.detections[0].label == sample[prediction_field].label
        )
        sample.save()

In [91]:
_evaluate_classification_modelr(view, 'resnet_pred')

 100% |███████████████| 1000/1000 [15.8s elapsed, 0s remaining, 62.6 samples/s]      


In [92]:
def _compute_classification_recall(patch_collection, label_field):
    eval_key = "eval_" + label_field
    counts = patch_collection.count_values(eval_key)
    tp, fn = counts.get(True, 0), counts.get(False, 0)
    recall = tp/float(tp + fn) if tp + fn > 0 else 0.0
    return recall

In [93]:
def get_concept_attr_classification_recall(dataset, label_field, concept, attributes):
    sub_patch_view = dataset.filter_labels("ground_truth", F("label") == concept)
    for attribute in attributes.items():
        if "skin_tone" in attribute[0]:
            sub_patch_view = sub_patch_view.filter_labels('ground_truth', F(f"skin_tone.{attribute[0]}") != 0)
        else:
            sub_patch_view = sub_patch_view.filter_labels('ground_truth', F(f"{attribute[0]}") == attribute[1])
    return _compute_classification_recall(sub_patch_view, label_field)

In [94]:
concept = 'lawman'
attributes = {"perceived_gender_presentation": "masc"}

get_concept_attr_classification_recall(view, 'resnet_pred', concept, attributes)

0.05194805194805195

In [67]:
view

Dataset:     FACET14
Media type:  image
Num samples: 25
Sample fields:
    id:               fiftyone.core.fields.ObjectIdField
    filepath:         fiftyone.core.fields.StringField
    tags:             fiftyone.core.fields.ListField(fiftyone.core.fields.StringField)
    metadata:         fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.metadata.ImageMetadata)
    ground_truth:     fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    coco_masks:       fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    yolov5m:          fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Detections)
    eval_yolov5m_tp:  fiftyone.core.fields.IntField
    eval_yolov5m_fp:  fiftyone.core.fields.IntField
    eval_yolov5m_fn:  fiftyone.core.fields.IntField
    resnet_pred:      fiftyone.core.fields.EmbeddedDocumentField(fiftyone.core.labels.Classification)
    eval_resnet_pred: fiftyone.core.fields.BooleanField
View stages:
    1.